In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pybedtools import BedTool
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os

In [3]:
def extract_binding_sequences(bed_file, fasta_file, output_fasta):

    genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    bed = BedTool(bed_file)

    records = []

    for i, interval in enumerate(bed):
        chrom = interval.chrom
        start = int(interval.start)
        end = int(interval.end)

        seq = genome[chrom].seq[start:end] 
        record = SeqRecord(seq, id=f"{chrom}:{start}-{end}", description="")
        records.append(record.upper())

    SeqIO.write(records, output_fasta, "fasta")
 

In [6]:
extract_binding_sequences("jun_np_chr22_GRCh38.bed", "chr22_GRCh28.fasta", "jun_np_chr22.fasta")

Ejercicio 5.2: 

a) genere los 3 sets de secuencias negativas. Para el conjunto N3 (seceencias
del genoma que NO unen el FT) puede en primer lugar tomar secuencias (de largo fijo)
adyacentes y/o a “n” nt de distancia del sitio de union del FP. Alternativamente puede
seleccionar secuencias del Chr22 al azar (o puede probar ambas).

b) escriba el codigo para a partir del fasta con las secuencias de ADN ls convierta en un tensor
de pytorch de dimension Nseqs, 4, largo Seq

c) integre al codigo “b” la opciòn de hacer “0” padding y/o recorte las secuencias a un largo fijo.

d) Escriba el codigo que genera ademas el tensor que posee la clasificaciòn real de la
secuencia en si une (y_true=1) (o no, y_true=0) el FT. Usualmente a este tensor se lo llama
y_true.

In [83]:
def one_hot_encoding(sequence):

    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    seq_len = len(sequence)
    encoding = np.zeros((4, seq_len), dtype=np.float32)

    indices = np.array([mapping.get(base, 4) for base in sequence])

    valid_indices = indices[indices < 4]  # Exclude 'N' positions
    encoding[valid_indices, np.arange(seq_len)[indices < 4]] = 1.0

    n_positions = (indices == 4)
    encoding[:, n_positions] = 0.25

    return torch.Tensor(encoding)


def dinucleotide_shuffle(sequence):
    
    dinucleotides = [sequence[i:i+2] for i in range(0, len(sequence), 2)]
    np.random.shuffle(dinucleotides)
    shuffled_sequence_str = ''.join(dinucleotides)
    
    return shuffled_sequence_str

def padding(one_hot_sequence, len = 343):

    L = one_hot_sequence.shape[1]
    pad_total = len - L
    pad_left = pad_total // 2
    pad_right = pad_total - pad_left
        
    pad_tensor = torch.nn.functional.pad(one_hot_sequence, (pad_left, pad_right), mode='constant', value=0)
   
    return pad_tensor


In [92]:
sites = list(SeqIO.parse("jun_np_chr22.fasta", "fasta"))
sites = [str(seq_record.seq) for seq_record in sites]
tn = [dinucleotide_shuffle(site) for site in sites]

sites_one_hot =[(one_hot_encoding(x)) for x in sites]
tn_one_hot = [(one_hot_encoding(x)) for x in tn]

sites_one_hot = np.array([(padding(x)) for x in sites_one_hot])
tn_one_hot = np.array([padding(x) for x in tn_one_hot])

y_true = np.array([1 for i in range(sites_one_hot.shape[0])] + [0 for i in range(tn_one_hot.shape[0])])

In [64]:
max([sites_one_hot[i].shape[1] for i in range(len(sites_one_hot))])

343